In [12]:

# Step 1: Define Sample Documents
documents = [
    {"section": "Employee Info", "content": "John's pay is processed on the 1st of every month."},
    {"section": "Employee Info", "content": "Mark is on a leave of absence until next Monday."},
    {"section": "Employee Info", "content": "Julie is a software engineer."},
    {"section": "Employee Info", "content": "Julie's pay is processed on the 1st of every month."},
    {"section": "Employee Info", "content": "Mark is a product manager."},
    {"section": "Employee Info", "content": "John is an AI architect and has salary of 500K USD."},
]

# Step 2: Get Content Texts
content_corpus = [doc["content"] for doc in documents]
content_corpus

["John's pay is processed on the 1st of every month.",
 'Mark is on a leave of absence until next Monday.',
 'Julie is a software engineer.',
 "Julie's pay is processed on the 1st of every month.",
 'Mark is a product manager.',
 'John is an AI architect and has salary of 500K USD.']

In [13]:
%pip install -q sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [14]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
doc_vectors = model.encode(content_corpus)

In [15]:
doc_vectors

array([[-0.04595948,  0.01248904,  0.02014634, ..., -0.07168268,
         0.06206652, -0.02307062],
       [ 0.01152449, -0.0042987 ,  0.06208021, ..., -0.03832173,
        -0.01415213,  0.04418526],
       [-0.04009407, -0.0479057 , -0.01047251, ...,  0.00029134,
         0.12071934, -0.00028944],
       [-0.04454852, -0.0229358 , -0.02192589, ..., -0.04162   ,
         0.09864228, -0.02970469],
       [-0.0095519 , -0.06924053,  0.03345447, ..., -0.08885367,
         0.07931168,  0.0559937 ],
       [-0.00961918,  0.00769998,  0.02414494, ..., -0.06450296,
         0.01247326, -0.02879707]], shape=(6, 384), dtype=float32)

In [16]:
# Step 3: User Query and Semantic Matching
import numpy as np

query = "Tell me about John's job role."
query_vec = model.encode([query])[0]
query_vec

array([-6.51649684e-02,  3.61150764e-02,  4.62175496e-02, -7.34472135e-03,
       -3.10224984e-02,  8.05633664e-02,  3.45326066e-02, -4.65819938e-03,
       -1.10947274e-01, -1.79800822e-03, -2.65288036e-02,  9.32842195e-02,
        2.82098576e-02,  1.08933551e-02, -7.03792572e-02, -1.04782647e-02,
       -1.89243830e-04,  5.41325249e-02, -2.91238800e-02, -7.49230757e-02,
        3.70897390e-02,  7.83124380e-03,  3.26573998e-02, -5.75900823e-02,
        1.00327563e-02, -4.39469330e-03, -4.21557343e-03,  5.77472858e-02,
       -4.81638759e-02, -7.59086162e-02, -3.83208506e-02, -9.13320854e-03,
        4.12793271e-02,  1.35801379e-02, -6.89527541e-02,  7.94207677e-02,
        1.30519113e-02,  2.58197337e-02,  1.66705791e-02, -4.17594612e-02,
        2.73222141e-02, -6.54361323e-02,  2.87316982e-02,  5.96740795e-03,
       -3.59649886e-03, -7.40957931e-02,  3.88318300e-02, -4.59655076e-02,
       -2.59812642e-02, -8.45156424e-03, -6.22111820e-02,  1.27522124e-03,
        2.08584778e-02, -

In [17]:
similarities = model.similarity(query_vec, doc_vectors)

# Ensure it's a 1D numpy array
similarities = np.asarray(similarities).squeeze()

# Now get top 3
top_3_indices = np.argsort(similarities)[::-1][:3]
top_scores = similarities[top_3_indices]

In [18]:
top_scores

array([0.46848616, 0.4624754 , 0.31589437], dtype=float32)

In [19]:
top_docs = [documents[i]['content'] for i in top_3_indices]

top_docs
context = "\n---\n".join(top_docs)
top_docs, context

(["John's pay is processed on the 1st of every month.",
  'John is an AI architect and has salary of 500K USD.',
  'Mark is a product manager.'],
 "John's pay is processed on the 1st of every month.\n---\nJohn is an AI architect and has salary of 500K USD.\n---\nMark is a product manager.")

In [20]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPEN_AI_API_KEY")

my_client = OpenAI(api_key=my_api_key)
# my_client

def ask_question_open_ai(prompt):

    # print(f"User asked: {prompt}")
    # my_client.chat.completions.create

    llm_response = my_client.chat.completions.create(
        model="gpt-5-nano",
        # messages=[
        #     {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
        #     {"role": "user", "content": prompt}
        # ]
        messages=[
            {"role": "system", "content": '''
             You are an assistant who answers only based on the given context.
             '''},
            {"role": "user", "content": f"Context: {context}\n\n User Question: {query}"} 
        ]

    )
    return llm_response.choices[0].message.content  


In [21]:
print (query)
response = ask_question_open_ai(query)

Tell me about John's job role.


In [22]:
print(f"User query: {query}")
print(f"Context: {context}")

print(f"\n\nOpen AI Response: {response}")

User query: Tell me about John's job role.
Context: John's pay is processed on the 1st of every month.
---
John is an AI architect and has salary of 500K USD.
---
Mark is a product manager.


Open AI Response: John's job role is AI architect.
